# Biostat M280 Homework 4

**Due June 12 @ 11:59PM**

In this homework, we build a classifer for handwritten digit recognition. Following figure shows example bitmaps of handwritten digits from U.S. postal envelopes. 

<img src="./handwritten_digits.png" width="400" align="center"/>

Each digit is represented by a $32 \times 32$ bitmap in which each element indicates one pixel with a value of white or black. Each $32 \times 32$ bitmap is divided into blocks of $4 \times 4$, and the number of white pixels are counted in each block. Therefore each handwritten digit is summarized by a vector $\mathbf{x} = (x_1, \ldots, x_{64})$ of length 64 where each element is a count between 0 and 16. 

We will use a model-based method by assuming a distribution on the count vector and carry out classification using probabilities. A common distribution for count vectors is the multinomial distribution. However as you will see in Q10, it is not a good model for handwritten digits. Let's work on a more flexible model for count vectors. In the Dirichlet-multinomial model, we assume the multinomial probabilities $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$, and density
$$
\begin{eqnarray*}
	\pi(\mathbf{p}) =  \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1},
\end{eqnarray*} 
$$
where $|\alpha|=\sum_{j=1}^d \alpha_j$.

## Q1

For a multivariate count vector $\mathbf{x}=(x_1,\ldots,x_d)$ with batch size $|\mathbf{x}|=\sum_{j=1}^d x_j$, show that the probability mass function for Dirichlet-multinomial distribution is
$$
    f(\mathbf{x} \mid \alpha) 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}  
    = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$
where $\Delta_d$ is the unit simplex in $d$ dimensions and $|\alpha| = \sum_{j=1}^d \alpha_j$.


### Answer to Q1  

According to the definition of probablity mass function, we can get the following:  
$$
f(\mathbf{x} \mid \alpha) 
    = \int_{\Delta_d}f(\mathbf{x} \mid \mathbf{p})\,f(\mathbf{p} \mid \alpha)\,d \mathbf{p} 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}   
$$
Then we will show it equals to the second part.  

$$
\begin{aligned}
f(\mathbf{x} \mid \alpha)
	&= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}\\
    &= 1
\end{aligned}
$$

## Q2

Given independent data points $\mathbf{x}_1, \ldots, \mathbf{x}_n$, show that the log-likelihood is
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$
Is the log-likelihood a concave function?

### Answer to Q2 

From Q1, we can get the probablity mass function. Since $(x_{1}, x_{2}, ..., x_{i})$ are independent, we can take log of the probablity mass function for each $x_{i}$ and take the summation. Then we can get the log-likelihood function: 



$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$

This is not a concave function, and it can be 

## Q3

Write Julia function to compute the log-density of the Dirichlet-multinomial distribution.

In [33]:
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    # your code here
    sum_x = sum(x)
    sum_α = sum(α)
    L = lgamma(sum_x + 1) - sum(lgamma.(x + 1)) + 
        sum(lgamma.(x + α)) - sum(lgamma.(α)) - 
        lgamma(sum_α + sum_x) + lgamma(sum_α)
    return L
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
end

dirmult_logpdf

In [1]:
using CSV

## Q4

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Each row contains the 64 counts of a digit and the last element (65th element) indicates what digit it is. For grading purpose, evaluate the total log-likelihood of this data at parameter values $\alpha=(1,\ldots,1)$ using your function in Q3.

In [ ]:
# Read digits data
digits = Matrix{Float64}(CSV.read("optdigits.tra", datarow = 1));
X = digits[:, 1:64]';
Y = digits[:, 65];

In [85]:
α = ones(64)
r = dirmult_logpdf(X, α)
sum(r)

-638817.993292528

## Q5

Derive the score function $\nabla L(\alpha)$, observed information matrix $-d^2L(\alpha)$, and Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ for the Dirichlet-multinomial distribution.

Comment on why Fisher scoring method is inefficient for computing MLE in this example.

### Answer to Q5  

The derivative of $\ln(\Gamma(z))$ is defined as the following:  

$$
 \Psi(z) = \frac{d\ln\Gamma(z)}{dz} = \frac{\Gamma(z)^{'}}{\Gamma(z)}
$$

$$
 \Psi^{'}(z) = \frac{\Gamma(z)^{''}}{\Gamma(z)} - (\frac{\Gamma(z)^{'}}{\Gamma(z)})^{2}
$$  

Then we can derive the score function $\nabla L(\alpha)$:  

$$
\Theta_{j}=\frac{\partial L(\alpha)}{\partial \alpha_{j}} 
= \sum_{i=1}^n[\Psi(\alpha_{j} + x_{ij}) - \Psi(\alpha_{j})] - \sum_{i=1}^n[\Psi(|\alpha| + |x_{i}|) - \Psi(|\alpha|)]
$$

$$
\nabla L(\alpha) = \begin{pmatrix}
\Theta_1\\
\Theta_2\\
...\\
\Theta_d
\end{pmatrix}\\
$$

And the observed information matrix: 


$$
-\frac{\partial ^{2} L(\alpha)}{\partial \alpha_{j} \alpha_{k}} = \begin{cases}
	\sum_{i=1}^n[\Psi'(|\alpha| + |x_{j}|) - \Psi'(|\alpha|)] & \text{if $j \neq k$} \\
	-\sum_{i=1}^n[\Psi'(\alpha_{j} + x_{ij}) - \Psi'(\alpha_{j})] + 
    \sum_{i=1}^n[\Psi'(|\alpha| + |x_{i}|) - \Psi'(|\alpha|)] & \text{if $j == k$}
	\end{cases}
$$   


$$
-d^2L(\alpha) = \begin{pmatrix}
\Theta'_{11} & \Theta'_{12} & ... & \Theta'_{1d}\\ 
\Theta'_{21} & \Theta'_{22} & ... & \Theta'_{2d}\\ 
... & ... & ... & ...\\ 
\Theta'_{d1} & \Theta'_{d2} & ... & \Theta'_{dd}
\end{pmatrix}
$$  

After obtaining the observed information matrix, we can simply plug in and get the Fisher information matrix:  

$$
\mathbf{E}[-d^2L(\alpha)] = \mathbf{E}[\Theta'_{jk}] = \sum\Theta'_{jk}p(\Theta)  
$$

To use the Fisher information matrix, we need to compute the expected value of the observed matrix, which is really difficult, so the Fisher scoring method is inefficient for this problem

## Q6

What structure does the observed information matrix possess that can facilitate the evaluation of the Newton direction? Is the observed information matrix always positive definite? What remedy can we take if it fails to be positive definite? (Hint: HW1 Q6.)

### Answer to Q6  

The observed information matrix can be decomposed as following
$$
-d^2L(\alpha) = \mathbf{A} +
  \sum_{i=1}^n[\Psi'(|\alpha| + |x_{i}|) - \Psi'(|\alpha|)]\mathbf{1}\mathbf{1}^{T}\\
  \text{where $\mathbf{A}$ is a diagonal matrix } \\
  \mathbf{A} = -\sum_{i=1}^n[\Psi'(\alpha_{j} + x_{ij}) - \Psi'(\alpha_{j})] 
$$

To make use the special structure of the observed information matrix, we can use the Woodbury formula as showed in HW1 Q6. 

We can denote $\mathbf{A}$ as the diagonal matrix, $\mathbf{B} = \sum_{i=1}^n[\Psi'(|\alpha| + |x_{i}|) - \Psi'(|\alpha|)]$, $\mathbf{U} = \mathbf{1}$, and  $\mathbf{V} = \mathbf{1}$. Then the observed information matrix can be written as:  

$$
-d^2L(\alpha) = \mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T
$$


We can see the $\mathbf{B}$ is a scalar b, and we can use $c$ to represent $-b$. Then if we want the observed information matrix to be positive definite, the inverse of the matrix should also be PD.  

Then inverse of the oberseved information matrix can be represented as the following:


$$
	(\mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} + \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1}
$$  

And it can be written as:  

$$
	(\mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} + \mathbf{A}^{-1} \mathbf{U}
    \frac{c}{1 - c\mathbf{V}^T \mathbf{A}^{-1}\mathbf{U}} \mathbf{V}^T \mathbf{A}^{-1}
$$  

So the observed information matrix is PD, if $\frac{c}{1 - c\mathbf{V}^T \mathbf{A}^{-1}\mathbf{U}} > 0$. If it fails to be PD, we can use the identity matrix $\mathbf{I}$ to replace it. 


## Q7

Discuss how to choose a good starting point. Implement this as the default starting value in your function below. (Hint: Method of moment estimator may furnish a good starting point.)  

Moments of Dirichelet distribution:

$$
E[p_j] = \frac{\alpha_j}{\sum_{i}\alpha_i}
$$  

$$
Var[p_j] = \frac{\alpha_j(|\alpha|-\alpha_j)}{(1+|\alpha|)|\alpha|^{2}}
$$  

Since $E[p_j^{2}] = Var[p_j] + [E[p_j]]^{2}$, $E[p_k^{2}]$ can be written as the following: 

$$
\begin{align*}
E[p_j^{2}] &= \frac{\alpha_j(|\alpha|-\alpha_j)}{(1+|\alpha|)|\alpha|^{2}} + \frac{\alpha_j}{\sum_{i}\alpha_i}\\
&=\frac{\alpha_{j}|\alpha|(1+\alpha_j)}{(1+|\alpha|)|\alpha|^{2}} \\
&=E[p_j]\frac{1+\alpha_j}{1+|\alpha|}
\end{align*}
$$  

Then for a given dataset $(x_1, x_2, ..., x_i)$, we can compute the moment estimator:  

$$
\hat{E}[p_j] = \frac{\sum_{i=1}^n\frac{x_{ij}}{|x_i|}}{n} \\
\hat{E}[p_j^2] = \frac{\sum_{i=1}^n(\frac{x_{ij}}{|x_i|})^2}{n}
$$   

We can denote $v = \sum_{j=1}^d\frac{\hat{E}[p_j^2]}{\hat{E}[p_j]}$, and then we can have:  

$$
v = \frac{d+|\alpha|}{1+|\alpha|} \\
|\alpha| = \frac{d-1}{1-v} \\
$$

$$
\begin{align*}
\alpha_j &= \hat{E}[p_j]|\alpha| \\
&= \frac{\sum_{i=1}^n\frac{x_{ij}}{|x_i|}}{n}|\alpha| \\
&= \frac{\sum_{i=1}^n\frac{x_{ij}}{|x_i|}}{n} \frac{d-1}{1-v}
\end{align*}
$$


## Q8

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using the Newton's method. 

In [1]:
"""
    dirmult_newton(X)

Find the MLE of Dirichlet-multinomial distribution using Newton's method.

# Argument
* `X`: an `n`-by-`d` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: a `d` vector of starting point (optional). 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
* `maximum`: the log-likelihood at MLE.   
* `estimate`: the MLE. 
* `gradient`: the gradient at MLE. 
* `hessian`: the Hessian at MLE. 
* `se`: a `d` vector of standard errors. 
* `iterations`: the number of iterations performed.
"""
function dirmult_newton(X::Matrix; α0::Vector = nothing, 
            maxiters::Int = 100, tolfun::Float64 = 1e-6)
    
    # set default starting point from Q7
    tot_n = size(X, 2)
    tot_d = size(X, 1)
    nonzero_indx = find(sum(X, 2) .!= 0)
    # rule zero zeros
    X = X[nonzero_indx, :]
    X_rowsum = sum(X, 1)
    
    n = size(X, 2)
    d = size(X, 1)
    
    n_iter = 0
    
    # If the α is no initialized, use the moment estimator as a starting point
    if α0 is nothing
        p_hat = X ./ X_rowsum
        v = sum(sum(p_hat.^2, 2) ./ sum(p_hat, 2))
        α0 = sum(p_hat, 2) / n * (d - v) / (v - 1)
    end
    
    # calculate the log likelihood of this data at the starting point
    loglold = sum(dirmult_logpdf(X, α0))
    dL = zeros(d, 1)
    inv_A = ones(d, 1)
    # Newton loop
    for iter in 1:maxiters
        # evaluate gradient (score)
        dL .= sum(broadcast(-, 
            digamma.(broadcast(+, X, α0)), digamma.(α0)), 2) - 
            sum(digamma.(broadcast(+, α0_sum, x_rowsum)) - 
            digamma(α0_sum))
        # compute Newton's direction
        # inv_A .= - Diagonal(squeeze(1 ./ sum(broadcast(-, 
            # trigamma.(broadcast(+, X, α0)), trigamma.(α0)), 2), 2))
        inv_A .= -1 ./ sum(broadcast(-, trigamma.(broadcast(+, X, α0)), trigamma.(α0)), 2)
        # compute scaling factor c
        c = - sum(trigamma.(broadcast(+, α0_sum, x_rowsum)) - 
            trigamma(α0_sum))
        # the threshold to determin whether matrix is PD
        thres = sum(inv_A)
        if c < 1 / thres
            c = c
        else
            c = 0.99 * 1 / thresh
        end
        # computer the direction
        direct = inv_A .* dL + (c / (1 - c * thres)) * inv_A * inv_A' * dL
        s_temp = - 0.99 * α0 ./ direct
        # ensure all steps are positive
        s_temp[find(s_temp .< 0)] = 10
        # get the minimum step
        s = minimum(s_temp)
        # line search loop
        for lsiter in 1:10
            # step halving
            s = s / 2
            α1 = α0 + s * direct
        end
        # compute the loglikelihood
        logl = sum(dirmult_logpdf(X, α1))
        
        # check convergence criterion
        if abs(logl - loglold) < tolfun * (abs(loglold) + 1)
            break;
        end
        loglold = logl
        α0 = α1
        n_iter += 1
    end
    
    # compute logl, gradient, Hessian from final iterate
    
    
    # output
    
end

dirmult_newton

## Q9

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Find the MLE for the subset of digit 0, digit 1, ..., and digit 9 separately using your function. 

In [5]:
#Pkg.add("SpecialFunctions")
#Pkg.update("SpecialFunctions")
using SpecialFunctions

In [3]:
# Read digits data
digits = Matrix{Float64}(CSV.read("optdigits.tra", datarow = 1));
X = digits[:, 1:64]';
Y = digits[:, 65];

In [6]:
tot_n = size(X, 2)
tot_d = size(X, 1)
nonzero_indx = find(sum(X, 2) .!= 0)
# rule zero zeros
X = X[nonzero_indx, :]
X_rowsum = sum(X, 1)
    
n = size(X, 2)
d = size(X, 1)
p_hat = X ./ X_rowsum
v = sum(sum(p_hat.^2, 2) ./ sum(p_hat, 2))
α0 = sum(p_hat, 2) / n * (d - v) / (v - 1)

62×1 Array{Float64,2}:
 0.0691224  
 1.26343    
 2.74168    
 2.66324    
 1.28098    
 0.325825   
 0.0332272  
 0.000446363
 0.448878   
 2.44655    
 2.7169     
 2.45151    
 1.90295    
 ⋮          
 2.25598    
 2.1313     
 0.865914   
 0.0342156  
 5.5286e-5  
 0.0646183  
 1.35259    
 2.77391    
 2.65573    
 1.54667    
 0.489726   
 0.0459873  

In [15]:
α0_sum

72.70729366814811

In [17]:
x_rowsum = sum(X, 1)

1×3823 Array{Float64,2}:
 303.0  367.0  317.0  305.0  266.0  …  274.0  276.0  328.0  348.0  255.0

In [20]:
# dL = sum(trigamma(X' + α0) - trigamma(α0), 2)
α0_sum = sum(α0)
x_rowsum = sum(X, 1)
dL = sum(broadcast(-, 
        digamma.(broadcast(+, X, α0)), digamma.(α0)), 2) - 
            sum(digamma.(broadcast(+, α0_sum, x_rowsum)) - 
            digamma(α0_sum))
# digamma(X[, :] + α0)
#(rowSums(digamma(t(x) + alpha0) - digamma(alpha0)) 
                #- sum(digamma(x_rowsum + alpha0_sum) -  digamma(alpha0_sum)))

62×1 Array{Float64,2}:
  2553.13  
  -137.561 
   275.733 
   230.906 
  -722.742 
 -2270.73  
 -2801.94  
   318.646 
  -661.797 
    29.4004
   366.71  
   242.062 
  -303.877 
     ⋮     
   102.371 
  -233.533 
 -1202.8   
  -228.358 
 11682.3   
  1392.78  
  -261.419 
   254.575 
   119.063 
  -550.961 
 -1891.74  
 -1652.18  

In [110]:
inv_A = - Diagonal(squeeze(1 ./ sum(broadcast(-, 
            trigamma.(broadcast(+, X, α0)), trigamma.(α0)), 2), 2))

62×62 Diagonal{Float64}:
 8.03757e-6   ⋅            ⋅           …   ⋅            ⋅        
  ⋅          0.000336749   ⋅               ⋅            ⋅        
  ⋅           ⋅           0.000749658      ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
  ⋅           ⋅            ⋅           …   ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
  ⋅           ⋅            ⋅           …   ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
  ⋅           ⋅            ⋅               ⋅            ⋅        
 ⋮                                     ⋱  ⋮                      
  ⋅           ⋅            ⋅           …   ⋅       

In [111]:
inv_A * dL

62×1 Array{Float64,2}:
  0.020521  
 -0.0463235 
  0.206705  
  0.168897  
 -0.278316  
 -0.245583  
 -0.0271067 
  2.11623e-5
 -0.0720317 
  0.0204857 
  0.266288  
  0.16104   
 -0.172488  
  ⋮         
  0.0636009 
 -0.149309  
 -0.332803  
 -0.00130926
  3.57076e-5
  0.0121094 
 -0.0978942 
  0.194467  
  0.0894328 
 -0.259726  
 -0.305542  
 -0.0168504 

In [112]:
inv_A = -1 ./ sum(broadcast(-, 
            trigamma.(broadcast(+, X, α0)), trigamma.(α0)), 2)
inv_A .* dL
# inv_A * inv_A'

62×1 Array{Float64,2}:
  0.020521  
 -0.0463235 
  0.206705  
  0.168897  
 -0.278316  
 -0.245583  
 -0.0271067 
  2.11623e-5
 -0.0720317 
  0.0204857 
  0.266288  
  0.16104   
 -0.172488  
  ⋮         
  0.0636009 
 -0.149309  
 -0.332803  
 -0.00130926
  3.57076e-5
  0.0121094 
 -0.0978942 
  0.194467  
  0.0894328 
 -0.259726  
 -0.305542  
 -0.0168504 

In [72]:
c = - sum(trigamma.(broadcast(+, α0_sum, x_rowsum)) - 
            trigamma(α0_sum))

42.98980958419122

In [70]:
thres = sum(inv_A)

0.02120684981161322

In [103]:
inv_ddL =  (inv_A + (c / (1 - c * thres)) * inv_A * inv_A') * dL

LoadError: [91mDimensionMismatch("dimensions must match")[39m

In [109]:
#inv_A = -1 ./ sum(broadcast(-, 
            #trigamma.(broadcast(+, X, α0)), trigamma.(α0)), 2)
inv_ddL = inv_A .* dL + (c / (1 - c * thres)) * inv_A * inv_A' * dL

62×1 Array{Float64,2}:
 -0.00350556 
 -1.05296    
 -2.03423    
 -2.01761    
 -1.42944    
 -0.568879   
 -0.0560258  
 -0.000177365
 -0.397393   
 -2.06239    
 -1.90439    
 -1.82768    
 -1.86927    
  ⋮          
 -1.79358    
 -2.0605     
 -1.15991    
 -0.0184479  
  2.65707e-5 
 -0.0138806  
 -1.2173     
 -2.08901    
 -2.15593    
 -1.66889    
 -0.788352   
 -0.0473379  

In [141]:
direct = inv_A .* dL + (c / (1 - c * thres)) * inv_A * inv_A' * dL
s_temp = - 0.99 * α0 ./ direct
# index = find(s_temp .< 0)
s_temp[find(s_temp .< 0)] = 10
s = minimum(s_temp)

0.5670226931876318

In [142]:
s = s / 2
α1 = α0 + s * direct

62×1 Array{Float64,2}:
 0.0681285  
 0.964905   
 2.16496    
 2.09122    
 0.875717   
 0.164542   
 0.0173432  
 0.000396078
 0.336213   
 1.86184    
 2.17698    
 1.93334    
 1.37299    
 ⋮          
 1.74748    
 1.54713    
 0.537067   
 0.0289854  
 6.28191e-5 
 0.0606829  
 1.00747    
 2.18165    
 2.0445     
 1.07353    
 0.266219   
 0.0325665  

In [31]:
# p_hat = X ./ X_rowsum
v = sum(p_hat.^2, 2) ./ sum(p_hat, 2)

62×1 Array{Float64,2}:
 0.00875826
 0.0298635 
 0.0428963 
 0.0427417 
 0.0365516 
 0.0312478 
 0.0251039 
 0.00992334
 0.0212868 
 0.0427079 
 0.0419354 
 0.0405339 
 0.0396795 
 ⋮         
 0.0403344 
 0.0413943 
 0.0326817 
 0.0131872 
 0.00290698
 0.0104981 
 0.0323324 
 0.0434933 
 0.0437035 
 0.0375223 
 0.0315339 
 0.0207589 

## Q10

As $\alpha / |\alpha| \to \mathbf{p}$, the Dirichlet-multinomial distribution converges to a multinomial with parameter $\mathbf{p}$. Therefore multinomial can be considered as a special Dirichlet-multinomial with $|\alpha|=\infty$. Perform a likelihood ratio test (LRT) whether Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively.

## Q11

Now we can construct a simple Bayesian rule for handwritten digits recognition:
$$
	\mathbf{x}	\mapsto \arg \max_k \widehat \pi_k f(x|\widehat \alpha_k).
$$
Here we can use the proportion of digit $k$ in the training set as the prior probability $\widehat \pi_k$. Report the performance of your classifier on the test set of 1797 digits in `optdigits.tes`.